# <span style='color:gray'> JunYeong Ahn / 2020047029 / Assignment2 </span>

---

#### <span style='background : yellow'> • use the Default data to classify default with income+balance+student </span>

-Here, I start with loading the essential packages.

In [83]:
library(ISLR)
library(MASS)
library(dplyr)
library(class)
library(boot)

In [84]:
summary(Default)

 default    student       balance           income     
 No :9667   No :7056   Min.   :   0.0   Min.   :  772  
 Yes: 333   Yes:2944   1st Qu.: 481.7   1st Qu.:21340  
                       Median : 823.6   Median :34553  
                       Mean   : 835.4   Mean   :33517  
                       3rd Qu.:1166.3   3rd Qu.:43808  
                       Max.   :2654.3   Max.   :73554  

#### <span style='background:yellow'> • split the data in tain and test with one random sample of 80/20 </span>
#### <span style='background:yellow'> • ? what do you have to consider here? </span>

-To split the original 'default' data into a train set and a test set, we should use sample() function who takes a sample of the specified size from the elements of x using either with or without replacement. Here, when I make a train set whose size is four-fifths the number of obsevation of Default without replacement, the rest of it naturally becomes a test set. Strictly speaking, 'train' is a collection of indices, not of direct values. <br>
If we do not draw the sample randomly, there's a chance that the sample becomes to be biased one, which also could lead to 'underfitting'. In addition, not to overfit or underfit, we should properly set the balance on the ratio of sample size to test size - here, 80/20.

In [85]:
set.seed(38)
train = sample(nrow(Default),size = nrow(Default)/(5/4))

# Logistic Regression
-The Logistic Regression is used to model the probability of a class Default / Not default. Each object being detected in the data would be assigned a probability between 0 and 1, with a sum of one.

-To fit the Logistic Regression model using our train set, I used glm() function who takes default for response and income & balance & student for predictors to take those predictors into consideration at once.

In [86]:
glm.fit <- glm(default ~ income + balance + student, data = Default, subset = train, family = "binomial")

#### <span style='background:yellow'> discuss the relevance of predictors </span>

-Using sunmmary() function, I tried to get the overall picture of my model.

In [87]:
summary(glm.fit)


Call:
glm(formula = default ~ income + balance + student, family = "binomial", 
    data = Default, subset = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.0956  -0.1451  -0.0583  -0.0215   3.7045  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.076e+01  5.501e-01 -19.561  < 2e-16 ***
income       4.457e-06  9.119e-06   0.489  0.62501    
balance      5.661e-03  2.602e-04  21.761  < 2e-16 ***
studentYes  -7.173e-01  2.615e-01  -2.743  0.00609 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2272.8  on 7999  degrees of freedom
Residual deviance: 1255.9  on 7996  degrees of freedom
AIC: 1263.9

Number of Fisher Scoring iterations: 8


-Seeing this, especially 'Coefficients' part, we can realize that the p-value of ***income*** predictor is conspicuously larger than
that of other predictors (and also ***income***'s p-value alone is greater than 0.05, which generally is the standard when deciding whether reject
the Null hypothesis or not). <br> To make it much sure before judging (by my eyes) that 'income' is trash as a predictor, I'll use step() function who chooses better model in a stepwise algorithm to check.

In [88]:
step(glm.fit, direction = "backward")

Start:  AIC=1263.9
default ~ income + balance + student

          Df Deviance    AIC
- income   1   1256.1 1262.1
<none>         1255.9 1263.9
- student  1   1263.3 1269.3
- balance  1   2264.3 2270.3

Step:  AIC=1262.14
default ~ balance + student

          Df Deviance    AIC
<none>         1256.1 1262.1
- student  1   1281.2 1285.2
- balance  1   2265.0 2269.0



Call:  glm(formula = default ~ balance + student, family = "binomial", 
    data = Default, subset = train)

Coefficients:
(Intercept)      balance   studentYes  
 -10.581678     0.005661    -0.814955  

Degrees of Freedom: 7999 Total (i.e. Null);  7997 Residual
Null Deviance:	    2273 
Residual Deviance: 1256 	AIC: 1262

-As the code above shows, when using step() function ***income*** was discarded, but ***balance*** and ***student*** have survived to be meaningful. <br>
Seeing the 'Coefficients' part, it turns out that ***balance*** has a positive effect (relationship) on ***default*** and ***student(Yes)*** has a negative one on ***default***. <br><br>
In a conclusion, although we've used and are going to use three predictors in our model, this doesn't mean that they are all of the importance; ***income*** and ***student*** predictors are necessary and important.

<span style='color:blue'>※ For the instructions <span style='background:yellow'> create a confusion matrix </span> and <span style='background:yellow'> calculate precision, recall and F1-score </span>, I repeat almost the same tasks three times more.
     Therefore, maybe there would be no further trivial explanations, of course except for special cases. ※ </span>

#### <span style='background:yellow'> create a confusion matrix </span>

-Before creating a confusion matrix, I predicted the response against the data Default[-train,], which corresponds to the test set;

Default[train,] = train set / Default[-train,] = test set.

In [89]:
glm.probs=predict(glm.fit, Default[-train,],type="response")

-Just checking what number No / Yes was converted into.

In [90]:
contrasts(Default$default)

,Yes
No,0
Yes,1


-Our test set has 2000 observations, so I made 'glm.pred', full of "No", whose length is 2000 too and change the value from "No" to "Yes" 
when the value of the prediction we made is higher than 0.5 (the middle point of 0("No") and 1("Yes")).

In [91]:
glm.pred=rep("No", 2000)
glm.pred[glm.probs>0.5]="Yes"

-Confusion matrix is a specific table layout that allows visualization of the performance of an algorithm. Here, this confusion matrix shows us four cases (2*2) represented by "Yes" or "No" of our prediction and true test set.

In [92]:
confusionmatrix_glm <-table(data.frame(pred = glm.pred, true = Default[-train,]$default))
confusionmatrix_glm

     true
pred    No  Yes
  No  1910   49
  Yes   14   27

#### <span style='background:yellow'> calculate precision, recall and F1-score </span>

Precision (the fraction of relevant instances among the retrieved instances) = True Positive / (True Positive + False Positive)

In [93]:
precision_glm <- confusionmatrix_glm[2,2] / sum(confusionmatrix_glm[2,])
precision_glm

[1] 0.6585366

Recall (the fraction of the total amount of relevant instances that were actually retrieved) = True Positive / (True Positive + False Negative)

In [94]:
recall_glm <- confusionmatrix_glm[2,2] / sum(confusionmatrix_glm[,2])
recall_glm 


[1] 0.3552632

F1-score is a measure of a test's accuracy, which is calculated from the precision and recall of the test through the harmonic mean of them.

In [95]:
F1score_glm <- 2*precision_glm*recall_glm/(precision_glm+recall_glm)
F1score_glm

[1] 0.4615385

# LDA
-LDA (Linear Discriminant Analysis) is a model that creates a decision boundary by learning the distribution of data and classifies data. This assumes that the observations from each class are drawn from a Gaussian distribution, and plugging estimates for the parameters into Bayes theorem in order to perform prediction.

-To fit the LDA model using our train set, I used lda() function who takes default for response and income & balance & student for predictors to take those predictors into consideration at once.

In [96]:
lda.fit <- lda(default ~ income + balance + student, data = Default, subset = train)

#### <span style='background:yellow'> discuss the relevance of predictors </span>

In [120]:
lda.fit

Call:
lda(default ~ income + balance + student, data = Default, subset = train)

Prior probabilities of groups:
      No      Yes 
0.967875 0.032125 

Group means:
      income   balance studentYes
No  33542.64  802.3658  0.2908433
Yes 32162.61 1728.7048  0.3735409

Coefficients of linear discriminants:
                     LD1
income      3.370464e-06
balance     2.243630e-03
studentYes -1.957734e-01

There is no way to do this task based on my current knowledge. <br>
1.) Simply using the summary() function to check p-value of each predictor by eyes doesn't work here. <br>
2.) Step() function also doesn't work well for LDA&QDA ; the code '?step' says that this function is mainly for lm() and glm(). <br>
3.) We've briefly learned about how to analyze our model through ANOVA which uses ***categorical independent variables*** and ***a continuous dependent variable***, but here our LDA&QDA model has ***continuous independent variables*** and ***a categorical dependent variable***. <br> <br>
So there might be a simple solution fuction for this task, but it is out of my knowledge. 

#### <span style='background:yellow'> create a confusion matrix </span>

-Predict the response against the data Default[-train,], which corresponds to the test set;

Default[train,] = train set / Default[-train,] = test set.

In [97]:
lda.pred <- predict(lda.fit, Default[-train,])

-Confusion matrix for my LDA model

In [98]:
confusionmatrix_lda <- table(data.frame(pred=lda.pred$class, true=Default[-train,]$default))
confusionmatrix_lda

     true
pred    No  Yes
  No  1916   54
  Yes    8   22

#### <span style='background:yellow'> calculate precision, recall and F1-score </span>

In [99]:
precision_lda <- confusionmatrix_lda[2,2] / sum(table(data.frame(lda.pred$class, Default[-train,]$default))[2,])
precision_lda

[1] 0.7333333

In [100]:
recall_lda <- confusionmatrix_lda[2,2] / sum(table(data.frame(lda.pred$class, Default[-train,]$default))[,2])
recall_lda 


[1] 0.2894737

In [101]:
F1score_lda <- 2*precision_lda*recall_lda/(precision_lda+recall_lda)
F1score_lda

[1] 0.4150943

# QDA
-QDA (Quadratic Discriminant Analysis) results from assuming that the observations from each class are drawn from a Gaussian distribution and that each class has its own covariance matrix, and plugging estimates for the parameters into Bayes theorem in order to perform prediction.

-To fit the QDA model using our train set, I used qda() function who takes default for response and income & balance & student for predictors to take those predictors into consideration at once.

In [102]:
qda.fit <- qda(default ~ income + balance + student, data = Default, subset = train)

#### <span style='background:yellow'> discuss the relevance of predictors </span>

-There's no way I can check this for the same reason with what I wrote about this part in LDA. 

#### <span style='background:yellow'> create a confusion matrix <span>

-Predict the response against the data Default[-train,], which corresponds to the test set;

Default[train,] = train set / Default[-train,] = test set.

In [103]:
qda.pred <- predict(qda.fit, Default[-train,])

-Confusion matrix for my QDA model

In [104]:
confusionmatrix_qda <- table(data.frame(pred=qda.pred$class, true=Default[-train,]$default))
confusionmatrix_qda

     true
pred    No  Yes
  No  1916   52
  Yes    8   24

#### <span style='background:yellow'> calculate precision, recall and F1-score </span>

In [105]:
precision_qda <- confusionmatrix_qda[2,2] / sum(table(data.frame(qda.pred$class, Default[-train,]$default))[2,])
precision_qda

[1] 0.75

In [106]:
recall_qda <- confusionmatrix_qda[2,2] / sum(table(data.frame(qda.pred$class, Default[-train,]$default))[,2])
recall_qda 


[1] 0.3157895

In [107]:
F1score_qda <- 2*precision_qda*recall_qda/(precision_qda+recall_qda)
F1score_qda

[1] 0.4444444

# KNN
-Non parametic method : no assumptions are made about the shape of the decision boundary. <br>
KNN takes a completely different approach
from the classifiers seen in this chapter. In order to make a prediction for
an observation X = x, the K training observations that are closest to x are
identified. Then X is assigned to the class to which the plurality of these
observations belong.

1. A matrix containing the predictors associated with the training data, labeled ***train_knn*** below.<br>
2. A matrix containing the predictors associated with the data for which
we wish to make predictions, labeled ***test_knn*** below.<br>
3. A vector containing the class labels for the training observations,
labeled ***train.default*** below.<br>
4. A value for K, the number of nearest neighbors to be used by the
classifier.

In [116]:
train_knn = cbind(Default$income, Default$balance, Default$student)[train,]
test_knn = cbind(Default$income, Default$balance, Default$student)[-train,]
train.default = Default$default[train]

-knn() forms predictions using a single command.

In [117]:
set.seed(3)
knn.pred = knn(train_knn, test_knn, train.default, k = 3)

#### <span style='background:yellow'> discuss the relevance of predictors </span>

Unlike Logistic Regression, KNN does not tell us numeric explanations on which predictors are important because it is non-parametic method; we don't get a table of
coefficients as in summary(glm.fit). <br>
So This part is also out of my current knowledge.

#### <span style='background:yellow'> create a confusion matrix </span>

In [119]:
confusionmatrix_knn <- table(data.frame(pred=knn.pred, true=Default[-train,]$default))
confusionmatrix_knn

     true
pred    No  Yes
  No  1907   60
  Yes   17   16

#### <span style='background:yellow'> calculate precision, recall and F1-score </span>

In [111]:
precision_knn <- confusionmatrix_knn[2,2] / sum(confusionmatrix_knn[2,])
precision_knn

[1] 0.4848485

In [112]:
recall_knn <- confusionmatrix_knn[2,2] / sum(confusionmatrix_knn[,2])
recall_knn 

[1] 0.2105263

In [113]:
F1score_knn <- 2*precision_knn*recall_knn/(precision_knn+recall_knn)
F1score_knn

[1] 0.293578

#### <span style='background:yellow'> discuss differences between the approaches <span>

We use ***Logistic regression*** if we were to use linear regression to predict a binary qualitative response some estimates could lie outside of the 0 to 1 interval, making them difficult to interpret as probabilities. <br>
However, we also use ***LDA(Linear Discriminant Analysis)*** for these reasons we've learned; 1. The paramater estimates for logistic regression tend to be unstable when the classes are well separated. LDA does not have this problem. 2. If the predictors have an approximately normal distribution for each class, and n is small, than LDA is more stable than logisitic regression. 3. LDA is more frequently used when there are more than two response classes.<br> ***QDA(Quadratic Discriminant Analysis)*** is similar with LDA in that it results from assuming that the observations from each class are drawn from a Gaussian distribution, and plugging estimates for the parameters into Bayes theorem in order to perform prediction. However, unlike LDA, ***QDA*** assumes that each class has its own covariance matrix. That is, it assumes that an observation from the kth class is of the form X ∼ N(μk, Σk), where Σk is a covariance matrix for the kth class. <br> As I saw in our mandatory text book, ***KNN(K-Nearest Neighbors)*** attempts to estimate the conditional distribution of Y given X, and then classify a given observation to the class with highest estimated probability. <br><br>
But there is no absolute ranking between them because they sometimes show strong performance but sometimes don't depending on how things are going. <br><br>For example, LDA is typically better than QDA with relatively few training observations because in this case variance of the classifier could be a major concern. Also, LDA is usually better than Logistic with assuming that the observations are drawn
from a Gaussian distribution with a common covariance matrix in each
class. But when this assumption is not true, Logistic could outperform LDA. Plus, Logistic and LDA who are seemingly very similar differ in that, in log odds, beta0 and beta1 in Logistic are estimated
using maximum likelihood, whereas c0 and c1 in LDA are computed using the estimated
mean and variance from a normal distribution,,,

#### <span style='background:yellow'> take the best approach and compare it with its own using a 10-fold cross-validation approach </span>

-Considering the F1 score of each four approach, ***Logistic regression*** outperformed the others in our case. (Logistic Regression : 0.46 > QDA : 0.44 > LDA : 0.41 > KNN : 0.3). Given this ranking, we could infer that our training observations were quite small, and that the observations are not drawn
from a Gaussian distribution with a common covariance matrix in each
class ***(Just inferrence from the ranking, never affects my objective output)***. Anyway, I'll conduct a 10-fold CV on my glm.fit model. <br> <br>
Cross-validation is a technique to evaluate predictive models by partitioning the original sample into a training set to train the model, and a test set to evaluate it. And ***10-fold CV*** involves randomly dividing the set of observations into 10 folds of approximately
equal size. The first fold is treated as a validation set, and the method
is fit on the remaining 9 folds. The mean squared error, MSE1, is
then computed on the observations in the held-out fold. This procedure is
repeated 10 times; each time, a different group of observations is treated
as a validation set. This process results in 10 estimates of the test error,
MSE1, MSE2,..., MSE10. The 10-fold CV estimate is computed by averaging
these values. <br><br> Here, with cv.glm() function, I conducted 10-fold CV aganinst our logistic model.

In [114]:
glm.fit <- glm(default ~ income + balance + student, data = Default, family = "binomial")
cv.err <- cv.glm(Default, glm.fit, K=10)
cv.err$delta[1]

[1] 0.02139544

-Training error, MSE obtained by the gap between the estimated value and true value, is calculated here.

In [115]:
mean((predict(glm.fit, new_data = Default[-train,], type = 'response') - (as.numeric(Default$default) - 1))^2 ) 

[1] 0.02129814

-Predicted MSE calculated from 10-fold CV has a tendency not to overestimate true test MSE unlike other general validation approaches.
<br> In my case, <u> MSE I calculated manually for certain test set (A)</u> is slightly lower than <u>10-fold CV's MSE (B)</u> but if I sample the train set agian and again (without set.seed(something)) (A) could become much higher or much lower whearas (B) is absolutely stable - without any change. Anyway, as this formula shows : MSE(Mean Squared Error) = sum((y-y hat)^2)/n, MSE gets lower as the model's predictions get closer to the answer(true). <br>

In a conclusion, MSEs, especially our 10-fold CV's MSE is very closer to 0, which means my model is decent no matter how the samples were drawn. 

Plus, through comparing how our model with changing train set predicted with MSE of 10-fold cross-validation, we could assure whether this model is biased or not. 